In [10]:
import typing
import pandas as pd
from tqdm import tqdm
import config
import src
import requests
import os
from dotenv import load_dotenv
load_dotenv("sjoerdAzure.env")  # Load environment variables from .env file
import time
import tqdm
import json
import numpy as np
import logging

import cltrier_lib as lib

from sklearn.metrics import cohen_kappa_score, classification_report
import krippendorff
import yaml

In [31]:
#set up helper variables and functions:
CFG = config.Config()

def load_json(path: str):
    with open(path, encoding='utf-8') as fp:
        return json.load(fp)
    
#set option variables:

#set options to low temperature (0,1):
options_low_str = """
seed: 42
temperature: 0.1
"""

options_low = yaml.safe_load(options_low_str)

MODELsmall: str = 'llama3.1:8b-instruct-q6_K' # options: 'gemma:7b-instruct-q6_K', 'gemma2:27b-instruct-q6_K', 'llama3.1:8b-instruct-q6_K', 'llama3.1:70b-instruct-q6_K', 'mistral:7b-instruct-v0.3-q6_K', 'mistral-large:123b-instruct-2407-q6_K', 'mixtral:8x7b-instruct-v0.1-q6_K', 'mixtral:8x22b-instruct-v0.1-q6_K', 'phi3:14b-medium-128k-instruct-q6_K' or 'qwen2:72b-instruct-q6_K'
MODELlarge: str = 'llama3.1:70b-instruct-q6_K' # options: 'gemma:7b-instruct-q6_K', 'gemma2:27b-instruct-q6_K', 'llama3.1:8b-instruct-q6_K', 'llama3.1:70b-instruct-q6_K', 'mistral:7b-instruct-v0.3-q6_K', 'mistral-large:123b-instruct-2407-q6_K', 'mixtral:8x7b-instruct-v0.1-q6_K', 'mixtral:8x22b-instruct-v0.1-q6_K', 'phi3:14b-medium-128k-instruct-q6_K' or 'qwen2:72b-instruct-q6_K'
MODELgpt4o = "nf-gpt-4o-2024-08-06" # in principe is er nu van elk model een nf (no filter) en een normale versie beschikbaar, de no filter versies zijn alleen voor onderzoekers beschikbaar voor analyze van content die niet door de filter heen zou komen.
MODELgpt4T = "nf-gpt-4-turbo" # Can be gpt-35-turbo, gpt-4-turbo, gpt-4 or Meta-Llama-3-8B-Instruct.
MODELgpt4 = "nf-gpt-4" # Can be gpt-35-turbo, gpt-4-turbo, gpt-4 or Meta-Llama-3-8B-Instruct.

options_zero_str = """
seed: 42
temperature: 0
"""
options_zero = yaml.safe_load(options_zero_str)

temperature_0 : int = 0
SEED: int = 42
MAX10: int = 10
TOPP1: int = 1


options_large_str = """
seed: 42
temperature: 0
num_predict: 2000
"""
options_large = yaml.safe_load(options_large_str)

#load environment variables:
api_key = os.environ.get('sjoerd_key')

#setttings:
api_endpoint = "https://ai-research-proxy.azurewebsites.net/chat/completions"
####### API REQUEST FORMATTING ######
headers = {
    "Content-Type": "application/json",
    "Authorization": "Bearer " + api_key
}#set up helper variables and functions:
CFG = config.Config()

def load_json(path: str):
    with open(path, encoding='utf-8') as fp:
        return json.load(fp)
    
#set option variables:

#set options to low temperature (0,1):
options_low_str = """
seed: 42
temperature: 0.1
"""

options_low = yaml.safe_load(options_low_str)

MODELsmall: str = 'llama3.1:8b-instruct-q6_K' # options: 'gemma:7b-instruct-q6_K', 'gemma2:27b-instruct-q6_K', 'llama3.1:8b-instruct-q6_K', 'llama3.1:70b-instruct-q6_K', 'mistral:7b-instruct-v0.3-q6_K', 'mistral-large:123b-instruct-2407-q6_K', 'mixtral:8x7b-instruct-v0.1-q6_K', 'mixtral:8x22b-instruct-v0.1-q6_K', 'phi3:14b-medium-128k-instruct-q6_K' or 'qwen2:72b-instruct-q6_K'
MODELlarge: str = 'llama3.1:70b-instruct-q6_K' # options: 'gemma:7b-instruct-q6_K', 'gemma2:27b-instruct-q6_K', 'llama3.1:8b-instruct-q6_K', 'llama3.1:70b-instruct-q6_K', 'mistral:7b-instruct-v0.3-q6_K', 'mistral-large:123b-instruct-2407-q6_K', 'mixtral:8x7b-instruct-v0.1-q6_K', 'mixtral:8x22b-instruct-v0.1-q6_K', 'phi3:14b-medium-128k-instruct-q6_K' or 'qwen2:72b-instruct-q6_K'
MODELgpt4o = "nf-gpt-4o-2024-08-06" # in principe is er nu van elk model een nf (no filter) en een normale versie beschikbaar, de no filter versies zijn alleen voor onderzoekers beschikbaar voor analyze van content die niet door de filter heen zou komen.
MODELgpt4T = "nf-gpt-4-turbo" # Can be gpt-35-turbo, gpt-4-turbo, gpt-4 or Meta-Llama-3-8B-Instruct.

options_zero_str = """
seed: 42
temperature: 0
"""
options_zero = yaml.safe_load(options_zero_str)

temperature_0 : int = 0
SEED: int = 42
MAX10: int = 10
TOPP1: int = 1


options_large_str = """
seed: 42
temperature: 0
num_predict: 2000
"""
options_large = yaml.safe_load(options_large_str)

#load environment variables:
api_key = os.environ.get('sjoerd_key')

#setttings:
api_endpoint = "https://ai-research-proxy.azurewebsites.net/chat/completions"
####### API REQUEST FORMATTING ######
headers = {
    "Content-Type": "application/json",
    "Authorization": "Bearer " + api_key
}

In [3]:
CFG.prompt_classify_files.items()

dict_items([('civility_jaidka', WindowsPath('data/prompts_classify/civility_jaidka.json')), ('constructiveness_jaidka', WindowsPath('data/prompts_classify/constructiveness_jaidka.json')), ('diversity_disagreement', WindowsPath('data/prompts_classify/diversity_disagreement.json')), ('diversity_disagreement_edit', WindowsPath('data/prompts_classify/diversity_disagreement_edit.json')), ('diversity_ideological_direction', WindowsPath('data/prompts_classify/diversity_ideological_direction.json')), ('diversity_positiondum', WindowsPath('data/prompts_classify/diversity_positiondum.json')), ('incivility_allcaps', WindowsPath('data/prompts_classify/incivility_allcaps.json')), ('incivility_attackreputation', WindowsPath('data/prompts_classify/incivility_attackreputation.json')), ('incivility_combine', WindowsPath('data/prompts_classify/incivility_combine.json')), ('incivility_jaidka', WindowsPath('data/prompts_classify/incivility_jaidka.json')), ('incivility_namecalling', WindowsPath('data/promp

In [4]:
pd.set_option('display.max_colwidth', 100) 

In [6]:
MODEL: str = 'llama3.1:70b-instruct-q6_K' # options: 'gemma:7b-instruct-q6_K', 'gemma2:27b-instruct-q6_K', 'llama3.1:8b-instruct-q6_K', 'llama3.1:70b-instruct-q6_K', 'mistral:7b-instruct-v0.3-q6_K', 'mistral-large:123b-instruct-2407-q6_K', 'mixtral:8x7b-instruct-v0.1-q6_K', 'mixtral:8x22b-instruct-v0.1-q6_K', 'phi3:14b-medium-128k-instruct-q6_K' or 'qwen2:72b-instruct-q6_K'
MODELsmall: str = 'llama3.1:8b-instruct-q6_K'

In [13]:
#load data
MHdata: pd.DataFrame = pd.read_csv('data/MH_BClemm_data/germany_val_all.csv') 
MHdata

,text,pol_label,neg_label,neg3_label,ideo_label,GPT_Pol1,GPT_Pol2,GPT_Neg1,GPT_Neg2,GPT_Neg_3Way1,GPT_Neg_3Way2,GPT_Ideo1,GPT_Ideo2,Pol_Recon,Neg_Recon,Neg3_Recon,Ideo_Recon
0,@thatleechi17 @KlemmChr @AndiScheuer @victorperli Ich würde sagen Sie vergleichen maximal kenntn...,1,1,0,1,1,1,1,1,0,0,1,1,1,1,0,1
1,"Glauben Sie nur nicht, dass Menschen die nicht dem Bundestag angehören, nichts zu sagen hätten. ...",1,0,1,1,1,1,0,0,1,1,1,1,1,0,1,1
2,Das ist ja das Mindeste. \r\n\r\nDass das aber das alles möglich ist trotz der Situation der Uig...,1,1,0,1,1,1,1,1,0,0,0,0,1,1,0,0
3,@Challen81690499 @c_lindner @rbrinkhaus @NowaboFM Na na na... TW,0,0,1,1,1,0,0,0,1,1,1,1,0,0,1,1
4,Ich sage: #DANKE!!!\r\nhttps://t.co/0pVneqXwTC\r\n#afd #btw #btw17 #btw2017,1,0,2,2,1,1,0,0,2,2,2,2,1,0,2,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
695,"@rehbergkk Ihr Denkfehler, in der Tat!",0,1,0,1,0,0,1,1,0,0,1,1,0,1,0,1
696,Die Bundesregierung spricht von „Einwanderung in die Sozialsysteme“ und erteilt gleichzeitig Arb...,1,1,0,0,1,1,0,1,0,0,0,0,1,1,0,0
697,Im #EU Parlament sitzen teilweise echt Verrückte https://t.co/wy2v0pYWLB,1,1,0,1,1,1,1,1,0,0,1,1,1,1,0,1
698,@StBrandner @M_HarderKuehnel Du natürlich! <U+0001F60E><U+0001F600>,0,0,1,1,0,0,0,0,2,2,1,1,0,0,2,1


In [7]:
#change options to low temperature (0,1) and compare result:
options_low = """
seed: 42
temperature: 0.1
"""

options_low = yaml.safe_load(options_low)

In [20]:
#the _label variables are the reconciled and definitive annotation values for the variables
#what does Llama3.1 predict for political orientation?
predictions: typing.Dict[str, pd.Series] = {
    label: (
        src.PromptClassify
        .from_json(path)
        (MHdata["text"], MODEL, options=options_low)
    )
    for label, path in CFG.prompt_classify_files.items() if label == 'political_ideology'
}

classifying political_ideology: 100%|██████████| 700/700 [49:31<00:00,  4.24s/it]  


In [9]:
predictions #note that these are based on llama3.1:70b-instruct-q6_K

{'political_ideology': 0           neutral
 1           neutral
 2           liberal
 3           neutral
 4      conservative
            ...     
 695         neutral
 696         liberal
 697    conservative
 698         neutral
 699         neutral
 Name: political_ideology, Length: 700, dtype: object}

In [21]:
#join to the dataset:
for _, preds in predictions.items():
    print(preds.value_counts())
    print("-" * 42)
    # Convert Series to DataFrame
    preds_df = preds.to_frame()
    # Rename the column in preds_df to 'Llama31_ideology'
    preds_df = preds_df.rename(columns={preds_df.columns[0]: 'Llama31_ideology_low'})
    MHdata = MHdata.join(preds_df)

political_ideology
neutral         474
liberal         145
conservative     81
Name: count, dtype: int64
------------------------------------------


In [11]:
#join to the dataset:
for _, preds in predictions.items():
    print(preds.value_counts())
    print("-" * 42)
    # Convert Series to DataFrame
    preds_df = preds.to_frame()
    # Rename the column in preds_df to 'Llama31_ideology'
    preds_df = preds_df.rename(columns={preds_df.columns[0]: 'Llama31_ideology'})
    MHdata = MHdata.join(preds_df)

political_ideology
neutral         454
liberal         159
conservative     87
Name: count, dtype: int64
------------------------------------------


In [23]:
#missing values in low temperature ideology:
MHdata.Llama31_ideology_low.isna().sum()

np.int64(0)

In [52]:
MHdata.Llama31_ideology.isna().sum()

np.int64(0)

In [12]:
#does Llama political ideology differ from manual political ideology ?
#make a crosstab to see if there are differences:
pd.crosstab(MHdata.loc[:, 'ideo_label'], MHdata.loc[:, 'Llama31_ideology'])

Llama31_ideology,conservative,liberal,neutral
ideo_label,,,
0,4,94,17
1,26,56,421
2,57,9,16


In [24]:
#does Llama political ideology low differ from manual political ideology ?
#make a crosstab to see if there are differences:
pd.crosstab(MHdata.loc[:, 'ideo_label'], MHdata.loc[:, 'Llama31_ideology_low'])

Llama31_ideology_low,conservative,liberal,neutral
ideo_label,,,
0,3,91,21
1,23,47,433
2,55,7,20


In [16]:
#convert the political ideology labels to numbers:
MHdata['Llama_ideo_score'] = MHdata['Llama31_ideology'].map({'liberal': 0, 'neutral': 1, 'conservative': 2})

In [25]:
MHdata['Llama_ideo_score_low'] = MHdata['Llama31_ideology_low'].map({'liberal': 0, 'neutral': 1, 'conservative': 2})

In [17]:
pd.crosstab(MHdata.loc[:, 'ideo_label'], MHdata.loc[:, 'Llama_ideo_score'])

Llama_ideo_score,0,1,2
ideo_label,,,
0,94,17,4
1,56,421,26
2,9,16,57


In [18]:
#and how does Llama3.1 perform compared to GPT
#make a crosstab to see if there are differences:
pd.crosstab(MHdata.loc[:, 'GPT_Ideo1'], MHdata.loc[:, 'Llama_ideo_score'])

Llama_ideo_score,0,1,2
GPT_Ideo1,,,
0,115,49,11
1,29,381,13
2,15,24,63


In [19]:
llm_human_column_pairs = [
    ("ideo_label", "Llama_ideo_score"),
    ("ideo_label", "GPT_Ideo1")
# ("TopicRelevance", "rationality_topic_relevance"),
]

In [20]:
print("Inter-coder reliablity: classification_report(human as gold, llm as predicted), cohen_kappa(human, llm):")

for human_col, llm_col in llm_human_column_pairs:
    subset = MHdata[[human_col, llm_col]].dropna()
    human = subset[human_col].tolist()
    llm = subset[llm_col].tolist()
    
    print("---")
    print(f"{classification_report(human, llm)}")
    print(f"cohen_kappa_score({human_col}): {cohen_kappa_score(human, llm)}")
    print(f"krippendorf({human_col}): {krippendorff.alpha(np.array([human, llm]), level_of_measurement="nominal")}")

Inter-coder reliablity: classification_report(human as gold, llm as predicted), cohen_kappa(human, llm):
---
              precision    recall  f1-score   support

           0       0.59      0.82      0.69       115
           1       0.93      0.84      0.88       503
           2       0.66      0.70      0.67        82

    accuracy                           0.82       700
   macro avg       0.72      0.78      0.75       700
weighted avg       0.84      0.82      0.82       700

cohen_kappa_score(ideo_label): 0.6206909689737066
krippendorf(ideo_label): 0.6192039671924076
---
              precision    recall  f1-score   support

           0       0.61      0.92      0.73       115
           1       0.96      0.81      0.88       503
           2       0.69      0.85      0.76        82

    accuracy                           0.83       700
   macro avg       0.75      0.86      0.79       700
weighted avg       0.87      0.83      0.84       700

cohen_kappa_score(ideo_label): 

In [ ]:
#Llama did ok, but GPT4 did better.

In [26]:
llm_human_column_pairs = [
    ("ideo_label", "Llama_ideo_score_low"),
#   ("ideo_label", "GPT_Ideo1")
# ("TopicRelevance", "rationality_topic_relevance"),
]
print("low temperature Inter-coder reliablity: classification_report(human as gold, llm as predicted), cohen_kappa(human, llm):")

for human_col, llm_col in llm_human_column_pairs:
    subset = MHdata[[human_col, llm_col]].dropna()
    human = subset[human_col].tolist()
    llm = subset[llm_col].tolist()
    
    print("---")
    print(f"{classification_report(human, llm)}")
    print(f"cohen_kappa_score({human_col}): {cohen_kappa_score(human, llm)}")
    print(f"krippendorf({human_col}): {krippendorff.alpha(np.array([human, llm]), level_of_measurement="nominal")}")

low temperature Inter-coder reliablity: classification_report(human as gold, llm as predicted), cohen_kappa(human, llm):
---
              precision    recall  f1-score   support

           0       0.63      0.79      0.70       115
           1       0.91      0.86      0.89       503
           2       0.68      0.67      0.67        82

    accuracy                           0.83       700
   macro avg       0.74      0.77      0.75       700
weighted avg       0.84      0.83      0.83       700

cohen_kappa_score(ideo_label): 0.6289335453713074
krippendorf(ideo_label): 0.6284897871397188


#very similar performance for low temperature english prompt of Llama3.1 compared to default temperature

In [8]:
#what if we translate the prompt first 
#what does Llama3.1 predict for political orientation?
predictions: typing.Dict[str, pd.Series] = {
    label: (
        src.PromptClassify
        .from_json(path)
        (MHdata["text"], model=MODEL, options=options_low)
    )
    for label, path in CFG.prompt_classify_files.items() if label == 'political_ideology_GER'
}

classifying political_ideology_GER: 100%|██████████| 700/700 [06:46<00:00,  1.72it/s]


In [9]:
#join to the dataset:
for _, preds in predictions.items():
    print(preds.value_counts())
    print("-" * 42)
    # Convert Series to DataFrame
    preds_df = preds.to_frame()
    # Rename the column in preds_df to 'Llama31_ideology'
    preds_df = preds_df.rename(columns={preds_df.columns[0]: 'Llama31_ideo_GER_low'})
    MHdata = MHdata.join(preds_df)

political_ideology_GER
Mitte     564
Links      83
Rechts     38
Name: count, dtype: int64
------------------------------------------


In [58]:
#join to the dataset:
for _, preds in predictions.items():
    print(preds.value_counts())
    print("-" * 42)
    # Convert Series to DataFrame
    preds_df = preds.to_frame()
    # Rename the column in preds_df to 'Llama31_ideology'
    preds_df = preds_df.rename(columns={preds_df.columns[0]: 'Llama31_ideo_GER'})
    MHdata = MHdata.join(preds_df)

political_ideology_GER
Mitte     505
Links      97
Rechts     48
Name: count, dtype: int64
------------------------------------------


In [ ]:
MHdata.Llama31_ideo_GER.isna().sum()    

In [11]:
MHdata.Llama31_ideo_GER_low.isna().sum()    

np.int64(15)

In [61]:
#convert the political ideology labels to numbers:
MHdata['Llama_ideoGER_score'] = MHdata['Llama31_ideo_GER'].map({'Links': 0, 'Mitte': 1, 'Rechts': 2})

In [12]:
#convert the political ideology labels to numbers:
MHdata['Llama_ideoGER_score_low'] = MHdata['Llama31_ideo_GER_low'].map({'Links': 0, 'Mitte': 1, 'Rechts': 2})

In [62]:
#does Llama political ideology differ from manual political ideology ?
#make a crosstab to see if there are differences:
pd.crosstab(MHdata.loc[:, 'ideo_label'], MHdata.loc[:, 'Llama_ideoGER_score'])

Llama_ideoGER_score,0.0,1.0,2.0
ideo_label,,,
0,59,51,0
1,32,421,11
2,6,33,37


In [13]:
#does Llama political ideology differ from manual political ideology ?
#make a crosstab to see if there are differences:
pd.crosstab(MHdata.loc[:, 'ideo_label'], MHdata.loc[:, 'Llama_ideoGER_score_low'])

Llama_ideoGER_score_low,0.0,1.0,2.0
ideo_label,,,
0,62,51,0
1,20,474,3
2,1,39,35


In [19]:
#percent agreement ideoGER_low:
100*571/685

83.35766423357664

In [63]:
#and compared to the english prompt:
pd.crosstab(MHdata.loc[:, 'Llama31_ideology'], MHdata.loc[:, 'Llama31_ideo_GER'])

Llama31_ideo_GER,Links,Mitte,Rechts
Llama31_ideology,,,
conservative,8,26,43
liberal,70,80,0
neutral,19,399,5


In [64]:
llm_human_column_pairs = [
    ("ideo_label", "Llama_ideoGER_score"),
    ("Llama_ideo_score", "Llama_ideoGER_score")
# ("TopicRelevance", "rationality_topic_relevance"),
]

In [14]:
llm_human_column_pairs = [
    ("ideo_label", "Llama_ideoGER_score_low"),
# ("TopicRelevance", "rationality_topic_relevance"),
]

In [15]:
print("Options low: Inter-coder reliablity: classification_report(human as gold, llm as predicted), cohen_kappa(human, llm):")

for human_col, llm_col in llm_human_column_pairs:
    subset = MHdata[[human_col, llm_col]].dropna()
    human = subset[human_col].tolist()
    llm = subset[llm_col].tolist()
    
    print("---")
    print(f"{classification_report(human, llm)}")
    print(f"cohen_kappa_score({human_col}): {cohen_kappa_score(human, llm)}")
    print(f"krippendorf({human_col}): {krippendorff.alpha(np.array([human, llm]), level_of_measurement="nominal")}")

Options low: Inter-coder reliablity: classification_report(human as gold, llm as predicted), cohen_kappa(human, llm):
---
              precision    recall  f1-score   support

           0       0.75      0.55      0.63       113
           1       0.84      0.95      0.89       497
           2       0.92      0.47      0.62        75

    accuracy                           0.83       685
   macro avg       0.84      0.66      0.72       685
weighted avg       0.83      0.83      0.82       685

cohen_kappa_score(ideo_label): 0.5580345014941591
krippendorf(ideo_label): 0.5540933207998926


considerably better reliability and less missing values than without low temperature setting, but still not as good as the english prompt

In [65]:
print("Inter-coder reliablity: classification_report(human as gold, llm as predicted), cohen_kappa(human, llm):")

for human_col, llm_col in llm_human_column_pairs:
    subset = MHdata[[human_col, llm_col]].dropna()
    human = subset[human_col].tolist()
    llm = subset[llm_col].tolist()
    
    print("---")
    print(f"{classification_report(human, llm)}")
    print(f"cohen_kappa_score({human_col}): {cohen_kappa_score(human, llm)}")
    print(f"krippendorf({human_col}): {krippendorff.alpha(np.array([human, llm]), level_of_measurement="nominal")}")

Inter-coder reliablity: classification_report(human as gold, llm as predicted), cohen_kappa(human, llm):
---
              precision    recall  f1-score   support

           0       0.61      0.54      0.57       110
           1       0.83      0.91      0.87       464
           2       0.77      0.49      0.60        76

    accuracy                           0.80       650
   macro avg       0.74      0.64      0.68       650
weighted avg       0.79      0.80      0.79       650

cohen_kappa_score(ideo_label): 0.5027665619859429
krippendorf(ideo_label): 0.5012600784626177
---
              precision    recall  f1-score   support

           0       0.72      0.47      0.57       150
           1       0.79      0.94      0.86       423
           2       0.90      0.56      0.69        77

    accuracy                           0.79       650
   macro avg       0.80      0.66      0.70       650
weighted avg       0.79      0.79      0.77       650

cohen_kappa_score(Llama_ideo_sc

In [ ]:
#unexpectedly, the german prompt did much worse than the english prompt.

In [66]:
#save data to parquet
MHdata.to_parquet('data/MH_BClemm_data/germany_val_all_llama.parquet')

In [41]:
#what if we adapt ideology labels further:
#what does Llama3.1 predict for political orientation?
predictions: typing.Dict[str, pd.Series] = {
    label: (
        src.PromptClassify
        .from_json(path)
        (MHdata["text"], MODEL)
    )
    for label, path in CFG.prompt_classify_files.items() if label == 'political_ideology_GER2'
}

classifying political_ideology_GER: 100%|██████████| 700/700 [12:55<00:00,  1.11s/it]


In [42]:
#join to the dataset:
for _, preds in predictions.items():
    print(preds.value_counts())
    print("-" * 42)
    # Convert Series to DataFrame
    preds_df = preds.to_frame()
    # Rename the column in preds_df to 'Llama31_ideology'
    preds_df = preds_df.rename(columns={preds_df.columns[0]: 'Llama31_ideo_GER2'})
    MHdata = MHdata.join(preds_df)

political_ideology_GER
Mittlere Mitte    420
Linksliberal      148
Konservativ        46
Name: count, dtype: int64
------------------------------------------


In [45]:
#convert the political ideology labels to numbers:
MHdata['Llama_ideoGER_score2'] = MHdata['Llama31_ideo_GER2'].map({'Linksliberal': 0, 'Mittlere Mitte': 1, 'Konservativ': 2})

In [46]:
MHdata['Llama_ideoGER_score2'].isna().sum()

np.int64(86)

In [47]:
#does Llama political ideology differ from manual political ideology ?
#make a crosstab to see if there are differences:
pd.crosstab(MHdata.loc[:, 'ideo_label'], MHdata.loc[:, 'Llama_ideoGER_score2'])

Llama_ideoGER_score2,0.0,1.0,2.0
ideo_label,,,
0,75,29,4
1,60,370,13
2,13,21,29


In [67]:
#and compared to the GER1 prompt:
pd.crosstab(MHdata.loc[:, 'Llama31_ideo_GER'], MHdata.loc[:, 'Llama31_ideo_GER2'])

Llama31_ideo_GER2,Konservativ,Linksliberal,Mittlere Mitte
Llama31_ideo_GER,,,
Links,7,63,20
Mitte,11,72,375
Rechts,27,3,5


In [68]:
llm_human_column_pairs = [
    ("ideo_label", "Llama_ideoGER_score2"),
    ("Llama_ideoGER_score", "Llama_ideoGER_score2")
# ("TopicRelevance", "rationality_topic_relevance"),
]

In [69]:
print("Inter-coder reliablity: classification_report(human as gold, llm as predicted), cohen_kappa(human, llm):")

for human_col, llm_col in llm_human_column_pairs:
    subset = MHdata[[human_col, llm_col]].dropna()
    human = subset[human_col].tolist()
    llm = subset[llm_col].tolist()
    
    print("---")
    print(f"{classification_report(human, llm)}")
    print(f"cohen_kappa_score({human_col}): {cohen_kappa_score(human, llm)}")
    print(f"krippendorf({human_col}): {krippendorff.alpha(np.array([human, llm]), level_of_measurement="nominal")}")

Inter-coder reliablity: classification_report(human as gold, llm as predicted), cohen_kappa(human, llm):
---
              precision    recall  f1-score   support

           0       0.51      0.69      0.59       108
           1       0.88      0.84      0.86       443
           2       0.63      0.46      0.53        63

    accuracy                           0.77       614
   macro avg       0.67      0.66      0.66       614
weighted avg       0.79      0.77      0.78       614

cohen_kappa_score(ideo_label): 0.5003894126262686
krippendorf(ideo_label): 0.4990361593355478
---
              precision    recall  f1-score   support

         0.0       0.46      0.70      0.55        90
         1.0       0.94      0.82      0.87       458
         2.0       0.60      0.77      0.68        35

    accuracy                           0.80       583
   macro avg       0.66      0.76      0.70       583
weighted avg       0.84      0.80      0.81       583

cohen_kappa_score(Llama_ideoGER

In [ ]:
#No improvement of performance.

In [51]:
#save data to parquet
MHdata.to_parquet('data/MH_BClemm_data/germany_val_all_llama.parquet')

In [29]:
#final try to improve German language prompt:
#what does Llama3.1 predict for political orientation?
predictions: typing.Dict[str, pd.Series] = {
    label: (
        src.PromptClassify
        .from_json(path)
        (MHdata["text"], MODEL, options=options_low)
    )
    for label, path in CFG.prompt_classify_files.items() if label == 'political_ideology_GER3'
}

classifying political_ideology_GER3: 100%|██████████| 700/700 [42:05<00:00,  3.61s/it]  


In [30]:
#join to the dataset:
for _, preds in predictions.items():
    print(preds.value_counts())
    print("-" * 42)
    # Convert Series to DataFrame
    preds_df = preds.to_frame()
    # Rename the column in preds_df to 'Llama31_ideology'
    preds_df = preds_df.rename(columns={preds_df.columns[0]: 'Llama31_ideo_GER3'})
    MHdata = MHdata.join(preds_df)

political_ideology_GER3
Moderate       513
Liberal         87
Konservativ     45
Name: count, dtype: int64
------------------------------------------


In [32]:
#convert the political ideology labels to numbers:
MHdata['Llama_ideoGER_score3'] = MHdata['Llama31_ideo_GER3'].map({'Liberal': 0, 'Moderate': 1, 'Konservativ': 2})

In [33]:
MHdata['Llama_ideoGER_score3'].isna().sum()

np.int64(55)

In [34]:
#does Llama political ideology differ from manual political ideology ?
#make a crosstab to see if there are differences:
pd.crosstab(MHdata.loc[:, 'ideo_label'], MHdata.loc[:, 'Llama_ideoGER_score3'])

Llama_ideoGER_score3,0.0,1.0,2.0
ideo_label,,,
0,59,46,0
1,23,441,9
2,5,26,36


In [35]:
llm_human_column_pairs = [
    ("ideo_label", "Llama_ideoGER_score3"),
#   ("Llama_ideoGER_score", "Llama_ideoGER_score2")
# ("TopicRelevance", "rationality_topic_relevance"),
]
print("Inter-coder reliablity: classification_report(human as gold, llm as predicted), cohen_kappa(human, llm):")

for human_col, llm_col in llm_human_column_pairs:
    subset = MHdata[[human_col, llm_col]].dropna()
    human = subset[human_col].tolist()
    llm = subset[llm_col].tolist()
    
    print("---")
    print(f"{classification_report(human, llm)}")
    print(f"cohen_kappa_score({human_col}): {cohen_kappa_score(human, llm)}")
    print(f"krippendorf({human_col}): {krippendorff.alpha(np.array([human, llm]), level_of_measurement="nominal")}")

Inter-coder reliablity: classification_report(human as gold, llm as predicted), cohen_kappa(human, llm):
---
              precision    recall  f1-score   support

           0       0.68      0.56      0.61       105
           1       0.86      0.93      0.89       473
           2       0.80      0.54      0.64        67

    accuracy                           0.83       645
   macro avg       0.78      0.68      0.72       645
weighted avg       0.82      0.83      0.82       645

cohen_kappa_score(ideo_label): 0.5639350973168099
krippendorf(ideo_label): 0.562640078693097


Best performing German Llama3.1 model so far, but did not reduce (but increased) number of missing classifications, and still performance is inferior to english language prompt

In [ ]:
#join to parquet data:
#save data to parquet
MHdatapar= pd.read_parquet('data/MH_BClemm_data/germany_val_all_llama.parquet')

In [42]:
MHdatapar.head()

,text,pol_label,neg_label,neg3_label,ideo_label,GPT_Pol1,GPT_Pol2,GPT_Neg1,GPT_Neg2,GPT_Neg_3Way1,...,Llama31_ideo_GER,Llama_ideoGER_score,Llama31_ideo_GER2,Llama_ideoGER_score2,Llama31_ideo_GER3,Llama_ideoGER_score3,Llama31_ideology_low,Llama_ideo_score_low,Llama31_ideo_GER_low,Llama_ideoGER_score_low
0,@thatleechi17 @KlemmChr @AndiScheuer @victorperli Ich würde sagen Sie vergleichen maximal kenntn...,1,1,0,1,1,1,1,1,0,...,Mitte,1.0,Mittlere Mitte,1.0,Moderate,1.0,neutral,1,Mitte,1.0
1,"Glauben Sie nur nicht, dass Menschen die nicht dem Bundestag angehören, nichts zu sagen hätten. ...",1,0,1,1,1,1,0,0,1,...,Mitte,1.0,Mittlere Mitte,1.0,Moderate,1.0,liberal,0,Mitte,1.0
2,Das ist ja das Mindeste. \r\n\r\nDass das aber das alles möglich ist trotz der Situation der Uig...,1,1,0,1,1,1,1,1,0,...,Mitte,1.0,Linksliberal,0.0,Moderate,1.0,liberal,0,Links,0.0
3,@Challen81690499 @c_lindner @rbrinkhaus @NowaboFM Na na na... TW,0,0,1,1,1,0,0,0,1,...,Mitte,1.0,Mittlere Mitte,1.0,Moderate,1.0,neutral,1,Mitte,1.0
4,Ich sage: #DANKE!!!\r\nhttps://t.co/0pVneqXwTC\r\n#afd #btw #btw17 #btw2017,1,0,2,2,1,1,0,0,2,...,Rechts,2.0,None,NaN,Konservativ,2.0,conservative,2,Rechts,2.0


In [41]:
MHdatapar = MHdatapar.join(MHdata.loc[:, ['Llama31_ideo_GER3', 'Llama_ideoGER_score3', 'Llama31_ideology_low', 'Llama_ideo_score_low', 'Llama31_ideo_GER_low' ,'Llama_ideoGER_score_low']])   


In [43]:
MHdatapar.to_parquet('data/MH_BClemm_data/germany_val_all_llama.parquet')

In [14]:
#what about Simon's more expansive english ideology prompt?
#what does Llama3.1 predict for political orientation?
predictions: typing.Dict[str, pd.Series] = {
    label: (
        src.PromptClassify
        .from_json(path)
        (MHdata["text"], MODEL, options=options_low)
    )
    for label, path in CFG.prompt_classify_files.items() if label == 'political_ideology_ext'
}

classifying political_ideology_ext: 100%|██████████| 700/700 [07:02<00:00,  1.66it/s]  


In [8]:
MHdatapar=pd.read_parquet('data/MH_BClemm_data/germany_val_all_llama.parquet')

In [18]:
MHdatapar.columns


Index(['text', 'pol_label', 'neg_label', 'neg3_label', 'ideo_label',
       'GPT_Pol1', 'GPT_Pol2', 'GPT_Neg1', 'GPT_Neg2', 'GPT_Neg_3Way1',
       'GPT_Neg_3Way2', 'GPT_Ideo1', 'GPT_Ideo2', 'Pol_Recon', 'Neg_Recon',
       'Neg3_Recon', 'Ideo_Recon', 'Llama31_ideology', 'Llama_ideo_score',
       'Llama31_ideo_GER', 'Llama_ideoGER_score', 'Llama31_ideo_GER2',
       'Llama_ideoGER_score2', 'Llama31_ideo_GER3', 'Llama_ideoGER_score3',
       'Llama31_ideology_low', 'Llama_ideo_score_low', 'Llama31_ideo_GER_low',
       'Llama_ideoGER_score_low'],
      dtype='object')

In [19]:
#join to the dataset:
for _, preds in predictions.items():
    print(preds.value_counts())
    print("-" * 42)
    # Convert Series to DataFrame
    preds_df = preds.to_frame()
    # Rename the column in preds_df to 'Llama31_ideology'
    preds_df = preds_df.rename(columns={preds_df.columns[0]: 'Llama31_ideo_EXT'})
    MHdata = MHdata.join(preds_df)

political_ideology_ext
Neutral    440
Left       162
Right       97
Name: count, dtype: int64
------------------------------------------


In [22]:
#convert the political ideology labels to numbers:
MHdata['Llama_ideoEXT_score'] = MHdata['Llama31_ideo_EXT'].map({'Left': 0, 'Neutral': 1, 'Right': 2})

In [23]:
MHdata['Llama_ideoEXT_score'].isna().sum()

np.int64(1)

In [24]:
llm_human_column_pairs = [
    ("ideo_label", "Llama_ideoEXT_score"),
#   ("Llama_ideoGER_score", "Llama_ideoGER_score2")
# ("TopicRelevance", "rationality_topic_relevance"),
]
print("Inter-coder reliablity: classification_report(human as gold, llm as predicted), cohen_kappa(human, llm):")

for human_col, llm_col in llm_human_column_pairs:
    subset = MHdata[[human_col, llm_col]].dropna()
    human = subset[human_col].tolist()
    llm = subset[llm_col].tolist()
    
    print("---")
    print(f"{classification_report(human, llm)}")
    print(f"cohen_kappa_score({human_col}): {cohen_kappa_score(human, llm)}")
    print(f"krippendorf({human_col}): {krippendorff.alpha(np.array([human, llm]), level_of_measurement="nominal")}")

Inter-coder reliablity: classification_report(human as gold, llm as predicted), cohen_kappa(human, llm):
---
              precision    recall  f1-score   support

           0       0.59      0.83      0.69       115
           1       0.93      0.82      0.87       503
           2       0.60      0.72      0.65        81

    accuracy                           0.81       699
   macro avg       0.71      0.79      0.74       699
weighted avg       0.84      0.81      0.82       699

cohen_kappa_score(ideo_label): 0.6110119022899242
krippendorf(ideo_label): 0.6086761112191166


In [25]:
#percent agreement
(0.59*115+0.93*503+0.60*81)/699


0.8358226037195995

In [ ]:
#very similar (but slightly worse) than the original shorter english prompt.

In [7]:
#how about zero temperature?
#change options to low temperature (0,1) and compare result:
options_zero = """
seed: 42
temperature: 0
"""

options_zero = yaml.safe_load(options_zero)

In [40]:
#the _label variables are the reconciled and definitive annotation values for the variables
#what does Llama3.1 predict for political orientation?
predictions: typing.Dict[str, pd.Series] = {
    label: (
        src.PromptClassify
        .from_json(path)
        (MHdata["text"], MODEL, options=options_zero)
    )
    for label, path in CFG.prompt_classify_files.items() if label in ['political_ideology', 'political_post', 'political_post_GER']
}

classifying political_post_GER: 100%|██████████| 700/700 [05:13<00:00,  2.23it/s]


In [41]:
predictions

{'political_post_GER': 0      political
 1      political
 2      political
 3      political
 4      political
          ...    
 695    political
 696    political
 697    political
 698    political
 699    political
 Name: political_post_GER, Length: 700, dtype: object}

In [42]:
#join to the dataset:
for _, preds in predictions.items():
    print(preds.value_counts())
    print("-" * 42)
    # Convert Series to DataFrame
    preds_df = preds.to_frame()
    # Rename the column in preds_df to 'Llama31_ideology'
    preds_df = preds_df.rename(columns={preds_df.columns[0]: 'Llama31_political_GER_zero'})
    MHdatapar = MHdatapar.join(preds_df)

political_post_GER
political        589
non-political    111
Name: count, dtype: int64
------------------------------------------


In [ ]:
#convert the political ideology labels to numbers:
MHdatapar['Llama31_ideology_zero_score'] = MHdatapar['Llama31_ideology_zero'].map({'liberal': 0, 'neutral': 1, 'conservative': 2})
MHdatapar['Llama31_political_zero_score'] = MHdatapar['Llama31_political_zero'].map({'non-political': 0, 'political': 1})
MHdatapar['Llama31_political_GER_zero_score'] = MHdatapar['Llama31_political_GER_zero'].map({'non-political': 0, 'political': 1})

In [25]:
#and how does Llama3.1 perform with zero compared to low temperature?
#make a crosstab to see if there are differences:
pd.crosstab(MHdatapar.loc[:, 'Llama31_ideology_zero_score'], MHdatapar.loc[:, 'Llama_ideo_score_low'])

Llama_ideo_score_low,0,1,2
Llama31_ideology_zero_score,,,
0,143,5,2
1,2,466,1
2,0,3,78


In [27]:
#and how did Llama3.1 perform in classifying political ?
pd.crosstab(MHdatapar.loc[:, 'Llama31_political_zero'], MHdatapar.loc[:, 'pol_label'])

pol_label,0,1
Llama31_political_zero,,
non-political,143,21
political,59,477


In [44]:
#does it matter whether we specify the German context in the Llama3.1 prompt when classifying political ?
pd.crosstab(MHdatapar.loc[:, 'Llama31_political_zero'], MHdatapar.loc[:, 'Llama31_political_GER_zero'])

Llama31_political_GER_zero,non-political,political
Llama31_political_zero,,
non-political,110,54
political,1,535


In [ ]:
#evaluete respective performance :
llm_human_column_pairs = [
    ("ideo_label", "Llama31_ideology_zero_score"),
    ("pol_label", "Llama31_political_zero_score"),
    ("pol_label", "Llama31_political_GER_zero_score"),
    ("pol_label", "GPT_Pol1")
]
print("Inter-coder reliablity: classification_report(human as gold, llm as predicted), cohen_kappa(human, llm):")

for human_col, llm_col in llm_human_column_pairs:
    subset = MHdatapar[[human_col, llm_col]].dropna()
    human = subset[human_col].tolist()
    llm = subset[llm_col].tolist()

    # Calculate percent agreement
    agreement = np.sum(np.array(human) == np.array(llm))/len(np.array(human)) * 100

    print("---")
    print(f"{classification_report(human, llm)}")
    print(f"cohen_kappa_score({human_col}): {cohen_kappa_score(human, llm)}")
    print(f"krippendorf({human_col}): {krippendorff.alpha(np.array([human, llm]), level_of_measurement="nominal")}")
    print(f"percent_agreement({human_col}): {agreement:.2f}%")

Inter-coder reliablity: classification_report(human as gold, llm as predicted), cohen_kappa(human, llm):
---
              precision    recall  f1-score   support

           0       0.62      0.81      0.70       115
           1       0.92      0.85      0.88       503
           2       0.69      0.68      0.69        82

    accuracy                           0.83       700
   macro avg       0.74      0.78      0.76       700
weighted avg       0.84      0.83      0.83       700

cohen_kappa_score(ideo_label): 0.6320606774080042
krippendorf(ideo_label): 0.6313698931428036
percent_agreement(ideo_label): 82.71%
---
              precision    recall  f1-score   support

           0       0.87      0.71      0.78       202
           1       0.89      0.96      0.92       498

    accuracy                           0.89       700
   macro avg       0.88      0.83      0.85       700
weighted avg       0.88      0.89      0.88       700

cohen_kappa_score(pol_label): 0.705176262477361

#llama3.1 beats GPT4 on classifying political posts, in German, even without mentioning this is German content referring to the German political system
#in fact, when we do mention the political context, performance fall slightly below that of GPT4...

In [47]:
#what is the effect of using a smaller model?
#what does Llama3.1 predict for political orientation and ideology?
predictions: typing.Dict[str, pd.Series] = {
    label: (
        src.PromptClassify
        .from_json(path)
        (MHdata["text"], model=MODELsmall, options=options_zero)
    )
    for label, path in CFG.prompt_classify_files.items() if label in ['political_ideology', 'political_post']
}

classifying political_post: 100%|██████████| 700/700 [03:00<00:00,  3.87it/s]


In [49]:
predictions

{'political_ideology': 0      conservative
 1           liberal
 2           liberal
 3           neutral
 4      conservative
            ...     
 695         neutral
 696         liberal
 697    conservative
 698         neutral
 699         neutral
 Name: political_ideology, Length: 700, dtype: object,
 'political_post': 0          political
 1          political
 2          political
 3      non-political
 4          political
            ...      
 695        political
 696        political
 697        political
 698    non-political
 699        political
 Name: political_post, Length: 700, dtype: object}

In [55]:
#join to the dataset:
for label, preds in predictions.items():
    print(preds.value_counts())
    print("-" * 42)
    # Convert Series to DataFrame
    preds_df = preds.to_frame()
    # Rename the column in preds_df to include the label
    preds_df.columns = [f'Llama31_{label}_8b']
    MHdatapar = MHdatapar.join(preds_df)

political_ideology
neutral         505
liberal         111
conservative     77
Name: count, dtype: int64
------------------------------------------
political_post
political        529
non-political    163
Name: count, dtype: int64
------------------------------------------


In [56]:
preds_df

,Llama31_political_post_8b
0,political
1,political
2,political
3,non-political
4,political
...,...
695,political
696,political
697,political
698,non-political


In [ ]:
#convert the political ideology labels to numbers:
MHdatapar['Llama31_ideology_8b_score'] = MHdatapar['Llama31_political_ideology_8b'].map({'liberal': 0, 'neutral': 1, 'conservative': 2})
MHdatapar['Llama31_political_8b_score'] = MHdatapar['Llama31_political_post_8b'].map()

In [58]:
#and how did Llama3.1 8b perform in classifying political ?
pd.crosstab(MHdatapar.loc[:, 'Llama31_political_ideology_8b'], MHdatapar.loc[:, 'ideo_label'])

ideo_label,0,1,2
Llama31_political_ideology_8b,,,
conservative,9,33,35
liberal,61,37,13
neutral,42,429,34


In [59]:
#and how did Llama3.1 8b perform in classifying political compared to large model ?
pd.crosstab(MHdatapar.loc[:, 'Llama31_political_ideology_8b'], MHdatapar.loc[:, 'Llama31_ideology_zero'])

Llama31_ideology_zero,conservative,liberal,neutral
Llama31_political_ideology_8b,,,
conservative,46,11,20
liberal,11,79,21
neutral,23,57,425


In [60]:
#and how did Llama3.1 8b perform in classifying political ?
pd.crosstab(MHdatapar.loc[:, 'Llama31_political_post_8b'], MHdatapar.loc[:, 'pol_label'])

pol_label,0,1
Llama31_political_post_8b,,
non-political,147,16
political,54,475


In [61]:
#and how did Llama3.1 8b perform in classifying political compared to large model ?
pd.crosstab(MHdatapar.loc[:, 'Llama31_political_post_8b'], MHdatapar.loc[:, 'Llama31_political_zero'])

Llama31_political_zero,non-political,political
Llama31_political_post_8b,,
non-political,126,37
political,37,492


In [63]:
#evaluete respective performance:
llm_human_column_pairs = [
    ("ideo_label", "Llama31_ideology_zero_score"),
    ("ideo_label", "Llama31_ideology_8b_score"),
    ("pol_label", "Llama31_political_zero_score"),
    ("pol_label", "Llama31_political_8b_score")
]
print("Inter-coder reliablity: classification_report(human as gold, llm as predicted), cohen_kappa(human, llm):")

for human_col, llm_col in llm_human_column_pairs:
    subset = MHdatapar[[human_col, llm_col]].dropna()
    human = subset[human_col].tolist()
    llm = subset[llm_col].tolist()

    # Calculate percent agreement
    agreement = np.sum(np.array(human) == np.array(llm))/len(np.array(human)) * 100

    print("---")
    print(f"{classification_report(human, llm)}")
    print(f"cohen_kappa_score({human_col}): {cohen_kappa_score(human, llm)}")
    print(f"krippendorf({human_col}): {krippendorff.alpha(np.array([human, llm]), level_of_measurement="nominal")}")
    print(f"percent_agreement({human_col}): {agreement:.2f}%")

Inter-coder reliablity: classification_report(human as gold, llm as predicted), cohen_kappa(human, llm):
---
              precision    recall  f1-score   support

           0       0.62      0.81      0.70       115
           1       0.92      0.85      0.88       503
           2       0.69      0.68      0.69        82

    accuracy                           0.83       700
   macro avg       0.74      0.78      0.76       700
weighted avg       0.84      0.83      0.83       700

cohen_kappa_score(ideo_label): 0.6320606774080042
krippendorf(ideo_label): 0.6313698931428036
percent_agreement(ideo_label): 82.71%
---
              precision    recall  f1-score   support

           0       0.55      0.54      0.55       112
           1       0.85      0.86      0.85       499
           2       0.45      0.43      0.44        82

    accuracy                           0.76       693
   macro avg       0.62      0.61      0.61       693
weighted avg       0.75      0.76      0.76     

In [ ]:
#the 8b model did worse than the 70b model for ideology, but better for political...

In [77]:
#what about GPT4o, GPT4Turbo and GPT4?

#classify messages using gpt4o:
MHGPTrun1 = ['political_ideology', 'political_post']
  

chunked_result: typing.List[pd.DataFrame] = []

for label, path in CFG.prompt_classify_files.items():
    if label in MHGPTrun1: 
        template = load_json(path).get('template')
        classes = load_json(path).get('classes')
        for index, row in tqdm.tqdm(MHdata["text"].items()):
            retry_count = 0
            max_retries = 10
            
            while retry_count < max_retries:
                try: 
                    response = requests.post(
                            url=api_endpoint,
                            headers=headers,
                            json={
                                'model': MODELgpt4,
                                'messages': [
                                    {
                                        "role": "system",
                                        "content": template
                                    },
                                    {
                                        "role": "user",
                                        "content": template.format(text=row)
                                    }
                                ],
                                'temperature': temperature_0,  
                                'seed': SEED,
                                "max_tokens": MAX10
                            }
                        )  

                    if response.status_code == 200:
                        data_response = response.json()
                        chunked_result.append(
                        pd.DataFrame(
                            data=[[index, classes.get(data_response["choices"][0]["message"]["content"], None)]],                                
                            columns=['index', label]
                            )
                        )
                        break  # Exit the retry loop on success
                    elif response.status_code == 429:
                        retry_count += 1
                        wait_time = 1 + (3 * retry_count * retry_count)
                        print(f"Rate limit exceeded. Retrying in {wait_time} seconds...")
                        print(response.text)
                        time.sleep(wait_time)
                    elif response.status_code == 500:
                        retry_count += 1
                        wait_time = 20
                        print(f"Failed to connect to API. Status code: {response.status_code}. Retrying in {wait_time} seconds...")
                        print(response.text)
                        time.sleep(wait_time)
                    else:
                        print(f"Failed to connect to API. Status code: {response.status_code}")
                        print(response.text)
                        break
                except requests.exceptions.RequestException as e:   
                    print(f"Failed to connect to API: {e}")
                    retry_count += 1
                    wait_time = 60
                    print(f"Retrying in {wait_time} seconds...")
                    time.sleep(wait_time)                 
                


classifications3 = pd.concat(chunked_result, ignore_index=True)
print(classifications3) 

0it [00:00, ?it/s]

4it [00:03,  1.21it/s]

Rate limit exceeded. Retrying in 4 seconds...
{"error":{"message":"litellm.RateLimitError: AzureException RateLimitError - Requests to the ChatCompletions_Create Operation under Azure OpenAI API version 2024-02-01 have exceeded call rate limit of your current OpenAI S0 pricing tier. Please retry after 7 seconds. Please go here: https://aka.ms/oai/quotaincrease if you would like to further increase the default rate limit.\nReceived Model Group=nf-gpt-4\nAvailable Model Group Fallbacks=None","type":null,"param":null,"code":"429"}}
Rate limit exceeded. Retrying in 13 seconds...
{"error":{"message":"litellm.RateLimitError: AzureException RateLimitError - Requests to the ChatCompletions_Create Operation under Azure OpenAI API version 2024-02-01 have exceeded call rate limit of your current OpenAI S0 pricing tier. Please retry after 3 seconds. Please go here: https://aka.ms/oai/quotaincrease if you would like to further increase the default rate limit.\nReceived Model Group=nf-gpt-4\nAvailab

8it [01:46, 11.69s/it]

Rate limit exceeded. Retrying in 4 seconds...
{"error":{"message":"litellm.RateLimitError: AzureException RateLimitError - Requests to the ChatCompletions_Create Operation under Azure OpenAI API version 2024-02-01 have exceeded call rate limit of your current OpenAI S0 pricing tier. Please retry after 8 seconds. Please go here: https://aka.ms/oai/quotaincrease if you would like to further increase the default rate limit.\nReceived Model Group=nf-gpt-4\nAvailable Model Group Fallbacks=None","type":null,"param":null,"code":"429"}}
Rate limit exceeded. Retrying in 13 seconds...
{"error":{"message":"litellm.RateLimitError: AzureException RateLimitError - Requests to the ChatCompletions_Create Operation under Azure OpenAI API version 2024-02-01 have exceeded call rate limit of your current OpenAI S0 pricing tier. Please retry after 4 seconds. Please go here: https://aka.ms/oai/quotaincrease if you would like to further increase the default rate limit.\nReceived Model Group=nf-gpt-4\nAvailab

...

Rate limit exceeded. Retrying in 4 seconds...
{"error":{"message":"litellm.RateLimitError: AzureException RateLimitError - Requests to the ChatCompletions_Create Operation under Azure OpenAI API version 2024-02-01 have exceeded call rate limit of your current OpenAI S0 pricing tier. Please retry after 8 seconds. Please go here: https://aka.ms/oai/quotaincrease if you would like to further increase the default rate limit.\nReceived Model Group=nf-gpt-4\nAvailable Model Group Fallbacks=None","type":null,"param":null,"code":"429"}}
Rate limit exceeded. Retrying in 13 seconds...
{"error":{"message":"litellm.RateLimitError: AzureException RateLimitError - Requests to the ChatCompletions_Create Operation under Azure OpenAI API version 2024-02-01 have exceeded call rate limit of your current OpenAI S0 pricing tier. Please retry after 4 seconds. Please go here: https://aka.ms/oai/quotaincrease if you would like to further increase the default rate limit.\nReceived Model Group=nf-gpt-4\nAvailab

696it [2:54:55, 12.43s/it]

Rate limit exceeded. Retrying in 4 seconds...
{"error":{"message":"litellm.RateLimitError: AzureException RateLimitError - Requests to the ChatCompletions_Create Operation under Azure OpenAI API version 2024-02-01 have exceeded call rate limit of your current OpenAI S0 pricing tier. Please retry after 9 seconds. Please go here: https://aka.ms/oai/quotaincrease if you would like to further increase the default rate limit.\nReceived Model Group=nf-gpt-4\nAvailable Model Group Fallbacks=None","type":null,"param":null,"code":"429"}}
Rate limit exceeded. Retrying in 13 seconds...
{"error":{"message":"litellm.RateLimitError: AzureException RateLimitError - Requests to the ChatCompletions_Create Operation under Azure OpenAI API version 2024-02-01 have exceeded call rate limit of your current OpenAI S0 pricing tier. Please retry after 4 seconds. Please go here: https://aka.ms/oai/quotaincrease if you would like to further increase the default rate limit.\nReceived Model Group=nf-gpt-4\nAvailab

700it [2:55:16, 15.02s/it]

      index political_ideology political_post
0         0            neutral            NaN
1         1            neutral            NaN
2         2            liberal            NaN
3         3            neutral            NaN
4         4       conservative            NaN
...     ...                ...            ...
1395    695                NaN  non-political
1396    696                NaN      political
1397    697                NaN      political
1398    698                NaN  non-political
1399    699                NaN  non-political

[1400 rows x 3 columns]


In [58]:
data_response

{'id': 'chatcmpl-Ahbb3pqIaEzZ3DHfv8F3dZxLZxV2D',
 'choices': [{'finish_reason': 'stop',
   'index': 0,
   'message': {'content': '1',
    'role': 'assistant',
    'tool_calls': None,
    'function_call': None}}],
 'created': 1734955989,
 'model': 'gpt-4',
 'object': 'chat.completion',
 'system_fingerprint': None,
 'usage': {'prompt_tokens': 1407,
  'completion_tokens': 1,
  'total_tokens': 1408},
 'service_tier': None}

In [15]:
political_post_gpt4o = classifications1.loc[:, ["index", "political_post"]].dropna()
political_post_gpt4o.set_index('index', drop=True, inplace=True)
MHdatapar = MHdatapar.join(political_post_gpt4o, rsuffix='_gpt4o')
political_ideology_gpt4o = classifications1.loc[:, ["index", "political_ideology"]].dropna()
political_ideology_gpt4o.set_index('index', drop=True, inplace=True)
MHdatapar = MHdatapar.join(political_ideology_gpt4o, rsuffix='_gpt4o')

MHdatapar = MHdatapar.rename(columns={
    'political_post': 'political_post_gpt4o',
    'political_ideology': 'political_ideology_gpt4o'
})

In [27]:
political_post_gpt4T = classifications2.loc[:, ["index", "political_post"]].dropna()
political_post_gpt4T.set_index('index', drop=True, inplace=True)
MHdatapar = MHdatapar.join(political_post_gpt4T, rsuffix='_gpt4T')
political_ideology_gpt4T = classifications2.loc[:, ["index", "political_ideology"]].dropna()
political_ideology_gpt4T.set_index('index', drop=True, inplace=True)
MHdatapar = MHdatapar.join(political_ideology_gpt4T, rsuffix='_gpt4T')

MHdatapar = MHdatapar.rename(columns={
    'political_post': 'political_post_gpt4T',
    'political_ideology': 'political_ideology_gpt4T'
})

In [78]:
political_post_gpt4 = classifications3.loc[:, ["index", "political_post"]].dropna()
political_post_gpt4.set_index('index', drop=True, inplace=True)
MHdatapar = MHdatapar.join(political_post_gpt4, rsuffix='_gpt4')
political_ideology_gpt4 = classifications3.loc[:, ["index", "political_ideology"]].dropna()
political_ideology_gpt4.set_index('index', drop=True, inplace=True)
MHdatapar = MHdatapar.join(political_ideology_gpt4, rsuffix='_gpt4')

MHdatapar = MHdatapar.rename(columns={
    'political_post': 'political_post_gpt4',
    'political_ideology': 'political_ideology_gpt4'
})

In [79]:
MHdatapar.head()

,text,pol_label,neg_label,neg3_label,ideo_label,GPT_Pol1,GPT_Pol2,GPT_Neg1,GPT_Neg2,GPT_Neg_3Way1,...,political_post_gpt4o_dum,political_conservative_gpt4o_dum,political_liberal_gpt4o_dum,political_ideology_gpt4o_score,political_post_gpt4T,political_ideology_gpt4T,political_post_gpt4T_dum,political_ideology_gpt4T_score,political_post_gpt4,political_ideology_gpt4
0,@thatleechi17 @KlemmChr @AndiScheuer @victorperli Ich würde sagen Sie vergleichen maximal kenntn...,1,1,0,1,1,1,1,1,0,...,1,0,0,1,political,neutral,1,1,political,neutral
1,"Glauben Sie nur nicht, dass Menschen die nicht dem Bundestag angehören, nichts zu sagen hätten. ...",1,0,1,1,1,1,0,0,1,...,1,0,0,1,political,neutral,1,1,political,neutral
2,Das ist ja das Mindeste. \r\n\r\nDass das aber das alles möglich ist trotz der Situation der Uig...,1,1,0,1,1,1,1,1,0,...,1,0,1,0,political,liberal,1,0,political,liberal
3,@Challen81690499 @c_lindner @rbrinkhaus @NowaboFM Na na na... TW,0,0,1,1,1,0,0,0,1,...,0,0,0,1,non-political,neutral,0,1,non-political,neutral
4,Ich sage: #DANKE!!!\r\nhttps://t.co/0pVneqXwTC\r\n#afd #btw #btw17 #btw2017,1,0,2,2,1,1,0,0,2,...,1,1,0,2,political,conservative,1,2,political,conservative


In [80]:
MHdatapar.loc[:, 'political_post_gpt4o_dum'] = MHdatapar.loc[:, 'political_post_gpt4o'].map({"political": 1, "non-political":0}).fillna(0).astype(int)
MHdatapar.loc[:, 'political_ideology_gpt4o_score'] = MHdatapar.loc[:, 'political_ideology_gpt4o'].map({'liberal': 0, 'neutral': 1, 'conservative': 2})
MHdatapar.loc[:, 'political_post_gpt4T_dum'] = MHdatapar.loc[:, 'political_post_gpt4T'].map({"political": 1, "non-political":0}).fillna(0).astype(int)
MHdatapar.loc[:, 'political_ideology_gpt4T_score'] = MHdatapar.loc[:, 'political_ideology_gpt4T'].map({'liberal': 0, 'neutral': 1, 'conservative': 2})
MHdatapar.loc[:, 'political_post_gpt4_dum'] = MHdatapar.loc[:, 'political_post_gpt4'].map({"political": 1, "non-political":0}).fillna(0).astype(int)
MHdatapar.loc[:, 'political_ideology_gpt4_score'] = MHdatapar.loc[:, 'political_ideology_gpt4'].map({'liberal': 0, 'neutral': 1, 'conservative': 2})



In [81]:
#evaluete respective performance:
llm_human_column_pairs = [
    ("ideo_label", "political_ideology_gpt4o_score"),
     ("ideo_label", "political_ideology_gpt4T_score"),
      ("ideo_label", "political_ideology_gpt4_score"),
    ("ideo_label", "GPT_Ideo1"),
    ("pol_label", "political_post_gpt4o_dum"),
    ("pol_label", "political_post_gpt4T_dum"),
    ("pol_label", "political_post_gpt4_dum"),
    ("pol_label", "GPT_Pol1")
]
print("Inter-coder reliablity: classification_report(human as gold, llm as predicted), cohen_kappa(human, llm):")

for human_col, llm_col in llm_human_column_pairs:
    subset = MHdatapar[[human_col, llm_col]].dropna()
    human = subset[human_col].tolist()
    llm = subset[llm_col].tolist()

    # Calculate percent agreement
    agreement = np.sum(np.array(human) == np.array(llm))/len(np.array(human)) * 100

    print("---")
    print(f"{classification_report(human, llm)}")
    print(f"cohen_kappa_score({human_col}): {cohen_kappa_score(human, llm)}")
    print(f"krippendorf({human_col}): {krippendorff.alpha(np.array([human, llm]), level_of_measurement="nominal")}")
    print(f"percent_agreement({human_col}): {agreement:.2f}%")

Inter-coder reliablity: classification_report(human as gold, llm as predicted), cohen_kappa(human, llm):
---
              precision    recall  f1-score   support

           0       0.64      0.88      0.74       115
           1       0.94      0.83      0.88       503
           2       0.63      0.76      0.69        82

    accuracy                           0.83       700
   macro avg       0.74      0.82      0.77       700
weighted avg       0.85      0.83      0.84       700

cohen_kappa_score(ideo_label): 0.6507203891972806
krippendorf(ideo_label): 0.6489503850713998
percent_agreement(ideo_label): 82.86%
---
              precision    recall  f1-score   support

           0       0.51      0.86      0.64       115
           1       0.94      0.79      0.86       503
           2       0.66      0.70      0.68        82

    accuracy                           0.79       700
   macro avg       0.71      0.78      0.73       700
weighted avg       0.84      0.79      0.80     

In [ ]:
#GPT4 in MH paper outperforms GPT4o and GPT4T on both variables, but also GPT4 on Azure...

In [82]:
#save data to parquet
MHdata.to_parquet('data/MH_BClemm_data/germany_val_all_llama.parquet')